In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn import preprocessing
%pylab inline


KeyError: "Couldn't find field google.protobuf.EnumDescriptorProto.EnumReservedRange.start"

In [4]:
#Search directory for all jpgs and returns a list
mylist = os.listdir("../1000subset/MorphII_cropped")
#Converts list into numpy array
mylist = np.array(mylist)
##Builds a an array of all zeros
lab = np.zeros((len(mylist),), dtype=np.int)
lab = np.resize(lab,(1000,2))

In [5]:
#Searchs strings in mylist for M (male) and returns value of 1 if exists and 0 for female if doesn't exist
for i in range(len(mylist)):
    if "M" in mylist[i]:
        lab[i,0] = 1
        lab[i,1] = 0
    else:
        lab[i,0] = 0
        lab[i,1] = 1


In [6]:
y_train = lab[:,0]
print(y_train.shape)

(1000,)


In [8]:
#Reading Features from CSV
features = pd.read_csv("../1000subset/morph2_partial.csv", header=None)
#Drops 1st Column
features  = features.drop(features.columns[[0]], axis=1)
features = features.values
print(features.shape)

(1000, 2568)


In [9]:
test_size = 0.2
testing_size = int(test_size*len(features))



train_x = np.array(features[:,0:2568][:-testing_size]).astype(np.float32)
train_y = np.array(lab[:,0][:-testing_size]).reshape(-1, 1).astype(np.float32)

test_x = np.array(features[1:,0:2568][-testing_size:]).astype(np.float32)
test_y = np.array(lab[:,0][-testing_size:]).reshape(-1, 1).astype(np.float32)

scaler = preprocessing.MinMaxScaler(feature_range=(-5,5))
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

print(train_x.shape,test_x.shape)


(800, 2568) (200, 2568)


In [10]:
import tensorflow as tf

config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_x)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=2,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)  # if TensorFlow >= 1.1
dnn_clf.fit(train_x, y_train, batch_size=50, steps=1000)


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114f53e48>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/k

SKCompat()

In [8]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(test_x)
accuracy_score(test_y, y_pred['classes'])

INFO:tensorflow:Restoring parameters from /var/folders/kt/nf1spxd56pbct1tf2r_nlmfh0000gn/T/tmp8kwty06f/model.ckpt-100000


0.63500000000000001

In [9]:
from sklearn.metrics import log_loss

y_pred_proba = y_pred['probabilities']
log_loss(test_y, y_pred_proba)

3.7497462030883084

In [10]:
#Find list of all filenames in directory
import glob
filelist = glob.glob('./1000subset/MorphII_cropped/*.JPG')
#Convert images into numpy arrays
raw = np.array([np.array(Image.open(fname)) for fname in filelist])
#Transforms the 3d array into a 2d array (1000,4200)
raw = raw.reshape((raw.shape[0],-1))
print(raw.shape)

(1000, 4200)


In [11]:
test_size = 0.2
testing_size = int(test_size*len(features))


train_x = np.array(raw[:,0:4200][:-testing_size]).astype(np.float32)
train_y = np.array(lab[:,0][:-testing_size]).reshape(-1, 1).astype(np.float32)

test_x = np.array(raw[1:,0:4200][-testing_size:]).astype(np.float32)
test_y = np.array(lab[:,0][-testing_size:]).reshape(-1, 1).astype(np.float32)

scaler = preprocessing.MinMaxScaler(feature_range=(-5,5))
scaler.fit(train_x)
train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

print(train_y.shape,test_y.shape)

(800, 1) (200, 1)


In [12]:
import tensorflow as tf

config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config

feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_x)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=2,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf)  # if TensorFlow >= 1.1
dnn_clf.fit(train_x, y_train, batch_size=50, steps=100000)


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10d0cd908>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/kt/nf1spxd56pbct1tf2r_nlmfh0000gn/T/tmpag85sxzl'}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supp

SKCompat()

In [13]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(test_x)
accuracy_score(test_y, y_pred['classes'])

INFO:tensorflow:Restoring parameters from /var/folders/kt/nf1spxd56pbct1tf2r_nlmfh0000gn/T/tmpag85sxzl/model.ckpt-100000


0.89000000000000001

In [14]:
from sklearn.metrics import log_loss

y_pred_proba = y_pred['probabilities']
log_loss(test_y, y_pred_proba)

0.92062019673935103

In [15]:
y_train = np.ndarray.flatten(train_y)
y_train.shape

(800,)

In [16]:
import tensorflow as tf

n_inputs = 60*70
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [17]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [18]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [19]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [20]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [21]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [22]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
##Need to fix eval function for in top K.  Not running with current label information.

In [23]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [24]:
n_epochs = 40
batch_size = 50

In [25]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: train_x, y: y_train})
        acc_train = accuracy.eval(feed_dict={X: train_x, y: y_train})
        acc_test = accuracy.eval(feed_dict={X: test_x,
                                            y: test_y})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")


InvalidArgumentError: targets must be 1-dimensional
	 [[Node: eval/InTopK = InTopK[T=DT_INT64, k=1, _device="/job:localhost/replica:0/task:0/cpu:0"](dnn/outputs/add, _arg_y_0_1)]]

Caused by op 'eval/InTopK', defined at:
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/d3r3zz3d/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).star

In [34]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = test_x[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [35]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", test_y[:20])

Predicted classes: [1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 0]
Actual classes:    [1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 0]


In [67]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [68]:
show_graph(tf.get_default_graph())

<iframe seamless style="width:1200px;height:620px;border:0" srcdoc="
        <script>
          function load() {
            document.getElementById(&quot;graph0.8194005496695869&quot;).pbtxt = 'node {\n  name: &quot;batch/Const&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_INT64\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_INT64\n        tensor_shape {\n          dim {\n            size: 1000\n          }\n          dim {\n            size: 2568\n          }\n        }\n        tensor_content: &quot;<stripped 20544000 bytes>&quot;\n      }\n    }\n  }\n}\nnode {\n  name: &quot;batch/Const_1&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_BOOL\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_BOOL\n        tensor_shape {\n        }\n        bool_val: true\n      }\n    }\n  }\n}\nnode {\n  name: &quot;batch/fifo_queue&quot;\n  op: &quot;FIFOQueueV2&quot;\n  attr {\n    key: &quot;capacity&quot;\n    value {\n      i: 32\n    }\n  }\n  attr {\n    key: &quot;component_types&quot;\n    value {\n      list {\n        type: DT_INT64\n      }\n    }\n  }\n  attr {\n    key: &quot;container&quot;\n    value {\n      s: &quot;&quot;\n    }\n  }\n  attr {\n    key: &quot;shapes&quot;\n    value {\n      list {\n        shape {\n          dim {\n            size: 1000\n          }\n          dim {\n            size: 2568\n          }\n        }\n      }\n    }\n  }\n  attr {\n    key: &quot;shared_name&quot;\n    value {\n      s: &quot;&quot;\n    }\n  }\n}\nnode {\n  name: &quot;batch/fifo_queue_enqueue&quot;\n  op: &quot;QueueEnqueueV2&quot;\n  input: &quot;batch/fifo_queue&quot;\n  input: &quot;batch/Const&quot;\n  attr {\n    key: &quot;Tcomponents&quot;\n    value {\n      list {\n        type: DT_INT64\n      }\n    }\n  }\n  attr {\n    key: &quot;timeout_ms&quot;\n    value {\n      i: -1\n    }\n  }\n}\nnode {\n  name: &quot;batch/fifo_queue_Close&quot;\n  op: &quot;QueueCloseV2&quot;\n  input: &quot;batch/fifo_queue&quot;\n  attr {\n    key: &quot;cancel_pending_enqueues&quot;\n    value {\n      b: false\n    }\n  }\n}\nnode {\n  name: &quot;batch/fifo_queue_Close_1&quot;\n  op: &quot;QueueCloseV2&quot;\n  input: &quot;batch/fifo_queue&quot;\n  attr {\n    key: &quot;cancel_pending_enqueues&quot;\n    value {\n      b: true\n    }\n  }\n}\nnode {\n  name: &quot;batch/fifo_queue_Size&quot;\n  op: &quot;QueueSizeV2&quot;\n  input: &quot;batch/fifo_queue&quot;\n}\nnode {\n  name: &quot;batch/Cast&quot;\n  op: &quot;Cast&quot;\n  input: &quot;batch/fifo_queue_Size&quot;\n  attr {\n    key: &quot;DstT&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;SrcT&quot;\n    value {\n      type: DT_INT32\n    }\n  }\n}\nnode {\n  name: &quot;batch/mul/y&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_FLOAT\n        tensor_shape {\n        }\n        float_val: 0.03125\n      }\n    }\n  }\n}\nnode {\n  name: &quot;batch/mul&quot;\n  op: &quot;Mul&quot;\n  input: &quot;batch/Cast&quot;\n  input: &quot;batch/mul/y&quot;\n  attr {\n    key: &quot;T&quot;\n    value {\n      type: DT_FLOAT\n    }\n  }\n}\nnode {\n  name: &quot;batch/fraction_of_32_full/tags&quot;\n  op: &quot;Const&quot;\n  attr {\n    key: &quot;dtype&quot;\n    value {\n      type: DT_STRING\n    }\n  }\n  attr {\n    key: &quot;value&quot;\n    value {\n      tensor {\n        dtype: DT_STRING\n        tensor_shape {\n        }\n        string_val: &quot;batch/fraction_of_32_full&quot;\n      }\n    }\n  }\n}\nnode {\n  name: &quot;batch/fraction_of_32_full&quot;\n  op: &quot;ScalarSummary&quot;\n  input: &quot;batch/fraction_of_32_full/tags&quot;\n  input: &quot;batch/mul&quot;\n  attr {\n  